In [1]:
import seaborn as sns
# from sklearn.model_selection import train_test_split
import gc
import os
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from collections import Counter
import matplotlib.pyplot as plt

%matplotlib inline
#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [44]:
result1 = pd.read_csv('./submit/submission_test2_0.75.csv')
result2 = pd.read_csv('./submit/dnn_0523_v1_7458.zip')
result3 = pd.read_csv('./submit/lgb.csv')

result1.shape
result2.shape
result3.shape

(2265879, 3)

(2265879, 3)

(2265879, 3)

In [45]:
result1.columns = ['aid', 'uid', 'score1']
result2.columns = ['aid', 'uid', 'score2']
result3.columns = ['aid', 'uid', 'score3']

result = result1.merge(result2, on=['aid', 'uid'], how='left')
result = result.merge(result3, on=['aid', 'uid'], how='left')

result.shape
result.head()

(2265879, 5)

,aid,uid,score1,score2,score3
0,1790,55374290,0.009278,0.089015,0.032487
1,1918,30704418,0.043932,0.132247,0.029739
2,1904,44454952,0.045259,0.145350,0.044198
3,562,30747797,0.050961,0.122601,0.054476
4,886,19138638,0.019318,0.099761,0.014037


## rank

In [46]:
result = result.sort_values(by=['score1'])
result = result.reset_index(drop=True)
result['rank1'] = result.index
result = result.sort_values(by=['score2'])
result = result.reset_index(drop=True)
result['rank2'] = result.index
result = result.sort_values(by=['score3'])
result = result.reset_index(drop=True)
result['rank3'] = result.index

In [47]:
result['score'] = 0.7 * result.rank1 + 0.15 * result.rank2 + 0.15 * result.rank3

result.score = (result.score - min(result.score)) / (max(result.score) - min(result.score))

In [48]:
result.head()

,aid,uid,score1,score2,score3,rank1,rank2,rank3,score
0,70,70221733,0.000907,0.009882,0.000349,9997,3136,0,0.003283
1,1672,34115737,0.001743,0.020209,0.000400,31140,28406,1,0.011488
2,70,1190861,0.000421,0.005852,0.000426,2163,316,2,0.000676
3,1379,27944300,0.001219,0.012238,0.000437,17111,6622,3,0.005712
4,70,79863929,0.000336,0.007383,0.000438,1282,940,4,0.000446


In [49]:
sub_zip_name = 'rank7'
result['score'] = result['score'].apply(lambda x: float('%.8f' % x))
result[['aid', 'uid', 'score']].to_csv('submission.csv', index=False)
os.system('zip -j '+sub_zip_name+'.zip submission.csv')
os.system('mv submission.csv ./submit/'+sub_zip_name+'.csv')
os.system('mv '+sub_zip_name+'.zip ./submit/')

0

0

0

## merge

In [7]:
def vote(x):
    if (x.score1 >= 0.5) & (x.score2 >= 0.5):
        return 2
    elif (x.score1 < 0.5) & (x.score2 < 0.5):
        return 0
    else:
        return 1

In [9]:
# result['vote'] = result.apply(vote, axis=1)

result.groupby(['vote']).aid.count()

vote
0    2253748
1       6767
2       5364
Name: aid, dtype: int64

In [14]:
def score(x):
    if x.vote == 2:
        return 0.9 * max(x.score1, x.score2) + 0.1 * min(x.score1, x.score2)
    elif x.vote == 0:
        return 0.1 * max(x.score1, x.score2) + 0.9 * min(x.score1, x.score2)
    else:
        return 0.8 * x.score1 + 0.2 * x.score2

In [15]:
result['score'] = result.apply(score, axis=1)

result.shape
result.head()

(2265879, 6)

,aid,uid,score1,score2,vote,score
0,1790,55374290,0.009278,0.089015,0,0.017251
1,1918,30704418,0.043932,0.132247,0,0.052763
2,1904,44454952,0.045259,0.145350,0,0.055268
3,562,30747797,0.050961,0.122601,0,0.058125
4,886,19138638,0.019318,0.099761,0,0.027363


In [17]:
sub_zip_name = 'vote82'
result['score'] = result['score'].apply(lambda x: float('%.8f' % x))
result[['aid', 'uid', 'score']].to_csv('submission.csv', index=False)
os.system('zip -j '+sub_zip_name+'.zip submission.csv')
os.system('mv submission.csv ./submit/'+sub_zip_name+'.csv')
os.system('mv '+sub_zip_name+'.zip ./submit/')

0

0

0

## lgb

In [2]:
import lightgbm as lgb

In [3]:
df = pd.read_csv('df_b.csv')

df.shape

(11064803, 156)

In [4]:
df['label'][df.label == -1] = 0

train = df[df.label != -99]
test = df[df.label == -99]

train.shape
test.shape

del df
gc.collect()

(8798814, 156)

(2265989, 156)

18

In [ ]:
by = ['aid', 'uid']
target = ['label']
predictors = list(set(train.columns) - set(by) - set(target))

In [ ]:
# lgb_cv
params = {
    'objective':'binary',
        # metric='binary_error',
        'num_leaves':7,
        'max_depth':3,
        'learning_rate':0.05,
        'reg_alpha' : .4,
        'reg_lambda' : .2,
        'random_state':1024,
        'colsample_bytree':0.8,
        'subsample':0.9,
#         'n_estimators':20000,
        'n_jobs': 31,
#         'device':'gpu',
        'histogram_pool_size':30270,
        'max_bin':63,
        'gpu_use_dp':True
}
        
len(predictors)

lgb_train = lgb.Dataset(train[predictors],label = train['label'])
print('Start training')
cv = lgb.cv(params,lgb_train,num_boost_round=10000,nfold=3,early_stopping_rounds=50,seed=1024,verbose_eval=100)

print('Baseline->' + str(len(cv['binary_logloss-mean'])) + ':' + str(cv['binary_logloss-mean'][-1]))

In [ ]:
lgb_online = lgb.LGBMClassifier(
        objective='binary',
        metric='binary_logloss',
        num_leaves=7,#35,
        max_depth=3,
        learning_rate=0.05, # small_eta
        reg_alpha = .4,
        reg_lambda = .2,
        colsample_bytree = .8,
        subsample = .9,
        random_state=1024,
        n_estimators=4000,
        n_jobs = 31,
        histogram_pool_size=30270,
        max_bin=63
    )
len(predictors)

submit_model = lgb_online.fit(train[predictors], train['label'])

test['score'] = submit_model.predict_proba(test[predictors])[:,1]
test['score'] = test['score'].apply(lambda x: float('%.8f' % x))
sub_zip_name = 'lgb'
test[['aid', 'uid','score']].to_csv('submission.csv', index=False)
os.system('zip -j '+sub_zip_name+'.zip submission.csv')
os.system('mv submission.csv ./submit/'+sub_zip_name+'.csv')
os.system('mv '+sub_zip_name+'.zip ./submit/')

153